# SDTW on Trainging with Unaligned Dataset
# Setting up Environment

```
$ python -m venv sdtw
$ source sdtw/bin/activate # for linux
$ ./sdtw/Scripts/activate # for windows
$ python.exe -m pip install --upgrade pip # upgrade pip
$ pip install -r requirements.txt
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.core.display as ipd
import pandas as pd
import os

from lib.nn_losses.soft_dtw_cuda import SoftDTW
from lib.nn_models.nn_dchord import dchord_pipeline


In [2]:
# Pipeline config

compression_params = {'gamma_init': 1.0,
                      'trainable': True}

temp_smooth_params = {'smoothing_type': 'weighted_sum', 
                      'avg_length': 51, 
                      'weight_init': 'uniform'}

# temp_smooth_params = {'smoothing_type': 'Gaussian', 
#                       'avg_length': 51, 
#                       'sigma_init': 50}

feature_norm_params = {'num_features': 12,
                       'norm': '2', 
                       'threshold': 1e-4}

chord_template_params = {'shared_weights': True, 
                         'initialize_parameters': True, 
                         'normalize_weights': False}

softmax_params = {'tau': 1, 
                  'trainable': False}

In [3]:
model = dchord_pipeline(compression_params=compression_params, temp_smooth_params=temp_smooth_params,
                        feature_norm_params=feature_norm_params, chord_template_params=chord_template_params,
                        softmax_params=softmax_params)

In [6]:
import torchinfo

torchinfo.summary(model, input_size=(1, 1, temp_smooth_params['avg_length'], 12), col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20)

Layer (type:depth-idx)                   Input Shape          Output Shape         Param #              Trainable
dchord_pipeline                          [1, 1, 51, 12]       [1, 1, 1, 24]        --                   Partial
├─log_compression: 1-1                   [1, 1, 51, 12]       [1, 1, 51, 12]       1                    True
├─temporal_smoothing: 1-2                [1, 1, 51, 12]       [1, 1, 1, 12]        --                   True
│    └─Conv2d: 2-1                       [1, 1, 51, 12]       [1, 1, 1, 12]        51                   True
├─feature_normalization: 1-3             [1, 1, 1, 12]        [1, 1, 1, 12]        --                   --
├─dchord_templates: 1-4                  [1, 1, 1, 12]        [1, 1, 1, 24]        --                   True
│    └─Conv2d: 2-2                       [1, 1, 1, 23]        [1, 2, 1, 12]        24                   True
├─softmax_temperature: 1-5               [1, 1, 1, 24]        [1, 1, 1, 24]        (1)                  False
Total params

In [13]:
normalize = False
sharp = False
sharp_weight_by_avg_alignment_sum = False


criterion = SoftDTW(use_cuda=False, gamma=1.0, dist_func=None, normalize=normalize)